In [ ]:
# Scripts Objectives:
# -Creata a table were each rows correspond to a different query sequences, 
# and each column indicates in which references the query sequence was mapped.
# -The table will be composed by the follwoing columns:
#   query_name, max_coverage, max_base_num_coverage, max_identity*, max_good_qaln_qaln_qaln_qaln_aln_qidentity**, 
#    Ref1.map_status† Ref1.max_coverage, Ref1.max_base_num_coverage, Ref1.max_identity
#    Ref2.map_status†, . . .  

In [2]:
from Bio import SeqIO
import pandas as pd
import os, re, sys

test = False
mito_test = True
# Replace 'your_fasta_file.fasta' with the path to your FASTA file
if test:
    input_query_fasta_path = '../toxo_reads/toxo1.trimmedReads.fasta'
    input_ref_fasta_dir_path = '../Data/map_contamination'
    output_unmapped_reads_path = '../Data/unmapped_reads.fasta'
    output_map_statistics = ''
elif mito_test:
    input_query_fasta_path = '../Blocks_Namasivayam.fa'
    input_ref_fasta_dir_path = '../Data/map_contamination'
    output_unmapped_reads_path = '../Data/mapped_blocks_Namasivayam.fasta'
    output_map_statistics = ''
else:
    input_query_fasta_path = sys.argv[1]
    input_ref_fasta_dir_path = sys.argv[2]
    output_unmapped_reads_path = sys.argv[3]
    output_map_statistics = sys.argv[4]

# Parse the FASTA file and extract the lengths
total_len = 0
num_records = 0
query_len = []
query_id = []
for record in SeqIO.parse(input_query_fasta_path, "fasta"):
    total_len += len(record.seq)
    num_records += 1
    query_len.append(len(record.seq))
    query_id.append(record.id)

paf_df = pd.DataFrame({'qname' : query_id, 'qlen': query_len})

print(f'Total reads length: {total_len}')
print(f'Sequence number: {num_records}')

Total reads length: 6216
Sequence number: 24


In [3]:
mito_test=False
test = True
if test:
    paf_file1_path = '../Data/map_contamination/TGVEG.paf'
    paf_file2_path = '../Data/map_contamination/TGA4.paf'
    paf_file3_path = '../Data/map_contamination/TgRH_pasteur.paf'
    paf_file4_path = '../Data/map_contamination/Chlorocebus_sabaeus.paf'
    paf_file5_path = '../Data/map_contamination/Micoplasma_tauri.paf'
    paf_file6_path = '../Data/map_contamination/Homo_sapiens.paf'
    paf_file6_path = '../Data/map_contamination/TG_APlast.paf'
    paf_file7_path = '../Data/map_contamination/Ncanium_LIV.paf'

    paf_files_path_list = [paf_file1_path, paf_file2_path, paf_file3_path, paf_file4_path, paf_file5_path, paf_file6_path, paf_file7_path]
elif mito_test:
    paf_file1_path = '../Data/map_contamination/TGVEG.mito.paf'
    paf_file2_path = '../Data/map_contamination/TGA4.mito.paf'
    paf_file3_path = '../Data/map_contamination/TgRH_pasteur.mito.paf'
    paf_file4_path = '../Data/map_contamination/Chlorocebus_sabaeus.mito.paf'
    paf_file5_path = '../Data/map_contamination/Micoplasma_tauri.mito.paf'
    paf_file6_path = '../Data/map_contamination/Homo_sapiens.mito.paf'
    paf_file6_path = '../Data/map_contamination/TG_APlast.mito.paf'
    paf_file7_path = '../Data/map_contamination/Ncanium_LIV.mito.paf'
    paf_files_path_list = [paf_file1_path, paf_file2_path, paf_file3_path, paf_file4_path, paf_file5_path, paf_file6_path, paf_file7_path]

else:
    paf_files_path_list = os.listdir(input_ref_fasta_dir_path)

# Define the path to your PAF file

# Define column names based on the PAF specification
column_names = [
    'qname', 'qlen', 'qstart', 'qend', 'strand', 'sname', 'slen', 'sstart',
    'send', 'matching_residues', 'aln_len', 'minimap_ql'
    ,'undf1','undf2','undf3','undf4','undf5','undf6']
    #'aln_type',	'minimizers',	'qscore',	'sscore',	'read_len', 'divergence']
    # qned - qlen: Shows the number of query bases that were mapped
    # aln_len: Shows the number of subject bases that were mapped

# Load the PAF file into a DataFrame
paf_df = pd.DataFrame({'qname' : query_id, 'qlen' : query_len})
for path in paf_files_path_list:

    df = pd.read_csv(path, sep='\t', header=None, names=column_names, index_col=None)
    if df.empty:
        print(f'{os.path.basename(path)} dataframe is empty')
        continue

    file_name = os.path.basename(path)
    org_name = re.sub('.[^.]+$', '', file_name)
    
    df['org_name'] = org_name
    df['coverage'] =(df['qend'] - df['qstart']).abs() / df['qlen']
    df['aln_qidentity'] = df['matching_residues'] / (df['qend'] - df['qstart']).abs()
    df['total_qidentity'] = df['matching_residues'] / df['qlen']
    df['undefined'] = df[['undf1','undf2','undf3','undf4','undf5','undf6']].astype(str).agg('|'.join, axis=1)
    
    pattern = r'(dv:f:(0|1)\.{0,1}[0-9]*)'
    df['divergence'] = df['undefined'].str.extract(pattern)[0]
    df['divergence'] = df['divergence'].str.replace(r'dv:f:' , '')
    df['divergence'] = df['divergence'].astype(float)
    
    # If a query has more than two alignaments (for the same reference),
    # select only the one with the greatest aln_qidentity
    max_total_qident = df.groupby('qname')['total_qidentity'].idxmax() 
    max_aln_qident = df.groupby('qname')['aln_qidentity'].idxmax() 
    max_qcover = df.groupby('qname')['coverage'].idxmax() 
    min_divergence = df.groupby('qname')['divergence'].idxmin() 

    df_total_qident = pd.DataFrame()
    df_aln_qident = pd.DataFrame()
    df_coverage = pd.DataFrame()
    df_divergence = pd.DataFrame()
    
    df_total_qident[['qname', f'{org_name}.max_total_qidentity']] = df.loc[max_aln_qident.values, ['qname','aln_qidentity']]
    df_aln_qident[['qname', f'{org_name}.max_aln_qidentity']] = df.loc[max_aln_qident.values, ['qname','aln_qidentity']]
    df_coverage[['qname', f'{org_name}.max_coverage']] = df.loc[max_qcover.values, ['qname','coverage']]
    df_divergence[['qname', f'{org_name}.min_divergence']] = df.loc[min_divergence.values, ['qname','divergence']]

    df_org = pd.merge(df_aln_qident, df_coverage, on='qname', how='outer')
    df_org = pd.merge(df_org, df_divergence, on='qname', how='outer')
    df_org = pd.merge(df_org, df_total_qident, on='qname', how='outer')

    paf_df = pd.merge(paf_df, df_org, on='qname', how='outer')

In [4]:
max_total_qident_cols = []
for col in paf_df.columns:
    if 'max_total_qidentity' in col:
        max_total_qident_cols.append(col)

paf_df[max_total_qident_cols] = paf_df[max_total_qident_cols].fillna(0)
paf_df['max_total_qidentity'] = paf_df[max_total_qident_cols].max(axis=1)

def high_low_identity(row):
    if row['max_total_qidentity'] > 0.90:
        return('HI')
    elif row['max_total_qidentity'] > 0.70 and row['max_total_qidentity'] <= 0.90 :
        return('MI')
    else:
        return('LI')
    
paf_df['ilevel'] = paf_df.apply(high_low_identity, axis=1)
paf_df['org_map'] = paf_df[max_total_qident_cols].idxmax(axis=1).str.replace('.max_total_qidentity', '')

In [6]:
#paf_df.sort_values('TGVEG.mito.max_aln_qidentity')

In [57]:
paf_df.sort_values('TGVEG.mito.max_aln_qidentity')

,qname,qlen,TGVEG.mito.max_aln_qidentity,TGVEG.mito.max_coverage,TGVEG.mito.min_divergence,TGVEG.mito.max_total_qidentity,TGA4.mito.max_aln_qidentity,TGA4.mito.max_coverage,TGA4.mito.min_divergence,TGA4.mito.max_total_qidentity,...,TgRH_pasteur.mito.max_coverage,TgRH_pasteur.mito.min_divergence,TgRH_pasteur.mito.max_total_qidentity,Ncanium_LIV.mito.max_aln_qidentity,Ncanium_LIV.mito.max_coverage,Ncanium_LIV.mito.min_divergence,Ncanium_LIV.mito.max_total_qidentity,max_total_qidentity,ilevel,org_map
23,Tg_mtDNA_V,161,0.475177,0.875776,0.0049,0.475177,0.581560,0.875776,0.0040,0.581560,...,0.875776,0.0036,0.723404,NaN,NaN,NaN,0.000000,0.723404,MI,TgRH_pasteur.mito
4,Tg_mtDNA_E,683,0.598616,0.846266,0.0664,0.598616,1.000000,0.995608,0.0003,1.000000,...,0.995608,0.0003,1.000000,0.394366,0.935578,0.0947,0.394366,1.000000,HI,TGA4.mito
5,Tg_mtDNA_F,179,0.609272,0.843575,0.0749,0.609272,1.000000,0.972067,0.0013,1.000000,...,0.972067,0.0013,1.000000,0.747899,0.709497,0.0141,0.747899,1.000000,HI,TGA4.mito
10,Tg_mtDNA_K,445,0.677725,0.474157,0.0588,0.677725,1.000000,0.995506,0.0004,1.000000,...,0.995506,0.0004,1.000000,0.801242,0.462921,0.0323,0.801242,1.000000,HI,TGA4.mito
8,Tg_mtDNA_I,204,0.862745,0.534314,0.0376,0.862745,1.000000,0.980392,0.0009,1.000000,...,0.980392,0.0009,1.000000,1.000000,0.406863,0.0036,1.000000,1.000000,HI,TGA4.mito
21,Tg_mtDNA_T,279,0.907895,0.706093,0.0297,0.907895,1.000000,0.985663,0.0008,1.000000,...,0.985663,0.0008,1.000000,0.790210,0.512545,0.0294,0.790210,1.000000,HI,TGA4.mito
2,Tg_mtDNA_C,1050,0.930233,0.301905,0.0267,0.930233,0.982456,0.977143,0.0031,0.982456,...,0.996190,0.0007,0.999044,1.000000,0.240952,0.0095,1.000000,1.000000,HI,Ncanium_LIV.mito
20,Tg_mtDNA_S,205,0.932292,0.936585,0.0251,0.932292,1.000000,0.975610,0.0010,1.000000,...,0.975610,0.0010,1.000000,0.947917,0.468293,0.0085,0.947917,1.000000,HI,TGA4.mito
22,Tg_mtDNA_U,354,0.971698,0.703390,0.0127,0.971698,1.000000,0.994350,0.0006,1.000000,...,0.994350,0.0006,1.000000,0.585799,0.954802,0.0737,0.585799,1.000000,HI,TGA4.mito
17,Tg_mtDNA_P,184,0.972222,0.586957,0.0148,0.972222,1.000000,0.961957,0.0012,1.000000,...,0.961957,0.0012,1.000000,0.895833,0.586957,0.0499,0.895833,1.000000,HI,TGA4.mito


In [48]:
print('Fraction of nucleotides mapped with high (>90%), medium (70-90%) and Low (<70%) identity')
display(paf_df.groupby('ilevel')['qlen'].sum()/total_len)

print('Number of nucleotides in sequences with high (>90%), medium (70-90%) and Low (<70%) identity')
display(paf_df.groupby('ilevel')['qlen'].sum())

print('Median size of sequences mapped with high (>90%), medium (70-90%) and Low (<70%) identity')
display(paf_df.groupby('ilevel')['qlen'].median())

print('Number of sequences with high (>90%), medium (70-90%) and Low (<70%) identity')
display(paf_df.groupby('ilevel')['qlen'].count())


hi_df = paf_df[paf_df['ilevel'] == 'HI']
mi_df = paf_df[paf_df['ilevel'] == 'MI']
li_df = paf_df[paf_df['ilevel'] == 'LI']

Fraction of nucleotides mapped with high (>90%), medium (70-90%) and Low (<70%) identity


ilevel
HI    0.967664
LI    0.006435
MI    0.025901
Name: qlen, dtype: float64

Number of nucleotides in sequences with high (>90%), medium (70-90%) and Low (<70%) identity


ilevel
HI    6015
LI      40
MI     161
Name: qlen, dtype: int64

Median size of sequences mapped with high (>90%), medium (70-90%) and Low (<70%) identity


ilevel
HI    181.5
LI     40.0
MI    161.0
Name: qlen, dtype: float64

Number of sequences with high (>90%), medium (70-90%) and Low (<70%) identity


ilevel
HI    22
LI     1
MI     1
Name: qlen, dtype: int64

Fraction of nucleotides mapped with high (>90%), medium (70-90%) and Low (<70%) identity
ilevel
HI    0.973684
LI    0.012504
MI    0.013812
Name: qlen, dtype: float64
Number of nucleotides in sequences with high (>90%), medium (70-90%) and Low (<70%) identity
ilevel
HI    1557456813
LI      20000564
MI      22092434
Name: qlen, dtype: int64
Median size of sequences mapped with high (>90%), medium (70-90%) and Low (<70%) identity
ilevel
HI    9292.0
LI    1372.0
MI    6342.0
Name: qlen, dtype: float64
Number of sequences with high (>90%), medium (70-90%) and Low (<70%) identity
ilevel
HI    121931
LI      4843
MI      2102
Name: qlen, dtype: int64

In [49]:
print('Reads mapped with high identity (>90%)')

print('Fraction of mapped nucleotides asigned to each reference')
display(hi_df.groupby('org_map')['qlen'].sum()/hi_df['qlen'].sum())

print('Median sequence size asigned to each reference')
display(hi_df.groupby('org_map')['qlen'].median())

Reads mapped with high identity (>90%)
Fraction of mapped nucleotides asigned to each reference


org_map
Ncanium_LIV.mito    0.174564
TGA4.mito           0.446052
TGVEG.mito          0.379385
Name: qlen, dtype: float64

Median sequence size asigned to each reference


org_map
Ncanium_LIV.mito    1050.0
TGA4.mito            204.5
TGVEG.mito           112.0
Name: qlen, dtype: float64

Reads mapped with high identity (>90%)
Fraction of mapped nucleotides asigned to each reference
org_map
Chlorocebus_sabaeus    0.016857
TGA4                   0.032556
TGVEG                  0.035012
TG_APlast              0.001050
TgRH_pasteur           0.914526
Name: qlen, dtype: float64
Median sequence size asigned to each reference
org_map
Chlorocebus_sabaeus    16879.0
TGA4                    8681.0
TGVEG                   6475.0
TG_APlast              19686.0
TgRH_pasteur            9368.5
Name: qlen, dtype: float64

In [39]:
print('Reads mapped with medium identity (70-90%)')

print('Fraction of mapped nucleotides asigned to each reference')
display(mi_df.groupby('org_map')['qlen'].sum()/mi_df['qlen'].sum())

print('Median sequence size asigned to each reference')
display(mi_df.groupby('org_map')['qlen'].median())

Reads mapped with medium identity (70-90%)
Fraction of mapped nucleotides asigned to each reference


org_map
TgRH_pasteur.mito   NaN
Name: qlen, dtype: float64

Median sequence size asigned to each reference


org_map
TgRH_pasteur.mito   NaN
Name: qlen, dtype: float64

Reads mapped with medium identity (70-90%)
Fraction of mapped nucleotides asigned to each reference
org_map
Chlorocebus_sabaeus    0.004433
TGA4                   0.215406
TGVEG                  0.317919
TgRH_pasteur           0.462243
Name: qlen, dtype: float64
Median sequence size asigned to each reference
org_map
Chlorocebus_sabaeus    3761.0
TGA4                   4688.0
TGVEG                  8199.0
TgRH_pasteur           7190.0
Name: qlen, dtype: float64

In [40]:
print('Reads mapped with low identity (<70%)')

print('Fraction of mapped nucleotides asigned to each reference')
display(li_df.groupby('org_map')['qlen'].sum()/li_df['qlen'].sum())

print('Median sequence size asigned to each reference')
display(li_df.groupby('org_map')['qlen'].median())

Reads mapped with low identity (<70%)
Fraction of mapped nucleotides asigned to each reference


org_map
TGVEG.mito    1.0
Name: qlen, dtype: float64

Median sequence size asigned to each reference


org_map
TGVEG.mito    8938.5
Name: qlen, dtype: float64

Reads mapped with low identity (<70%)
Fraction of mapped nucleotides asigned to each reference
org_map
Chlorocebus_sabaeus    0.566965
TGA4                   0.122387
TGVEG                  0.278444
TgRH_pasteur           0.032203
Name: qlen, dtype: float64
Median sequence size asigned to each reference
org_map
Chlorocebus_sabaeus    9180.0
TGA4                   7804.0
TGVEG                  1268.0
TgRH_pasteur           8966.0
Name: qlen, dtype: float64

In [254]:
li_ids = set(li_df['qname'])
with open(output_unmapped_reads_path, 'w') as file:
    for record in SeqIO.parse(input_query_fasta_path, "fasta"):
        if record.id in li_ids:
            seq_head=f'>{record.id}\n'
            seq_body=f'{record.seq}\n'
            file.write(seq_head)
            file.write(seq_body)


# Section with test/squetch code

In [41]:
paf_df_sort = paf_df.filter(regex='qlen|min_divergence|max_coverage').sort_values(by='qlen',ascending=False)
paf_df_sort = paf_df_sort.reset_index()
paf_df_sort.head(6)

,index,qlen,TGVEG.mito.max_coverage,TGVEG.mito.min_divergence,TGA4.mito.max_coverage,TGA4.mito.min_divergence,TgRH_pasteur.mito.max_coverage,TgRH_pasteur.mito.min_divergence,Ncanium_LIV.mito.max_coverage,Ncanium_LIV.mito.min_divergence
0,51912,108517.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,104371,105404.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,22719,99502.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28700,96911.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,32082,96252.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,42046,95866.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
paf_df.sort_values(by='qlen',ascending=False)


,qname,qlen,TGVEG.mito.max_aln_qidentity,TGVEG.mito.max_coverage,TGVEG.mito.min_divergence,TGVEG.mito.max_total_qidentity,TGA4.mito.max_aln_qidentity,TGA4.mito.max_coverage,TGA4.mito.min_divergence,TGA4.mito.max_total_qidentity,...,TgRH_pasteur.mito.max_coverage,TgRH_pasteur.mito.min_divergence,TgRH_pasteur.mito.max_total_qidentity,Ncanium_LIV.mito.max_aln_qidentity,Ncanium_LIV.mito.max_coverage,Ncanium_LIV.mito.min_divergence,Ncanium_LIV.mito.max_total_qidentity,max_total_qidentity,ilevel,org_map
51912,6743d56d-6b55-42f8-898d-ca6ac80a5031,108517.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.00000,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,LI,TGVEG.mito
104371,cf81ab03-c55c-4fa1-9969-a91722b59b2b,105404.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.00000,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,LI,TGVEG.mito
22719,2d299152-0621-4079-b43b-7da225d6eb95,99502.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.00000,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,LI,TGVEG.mito
28700,3910ae3b-a42d-4096-9e19-9b632217ec63,96911.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.00000,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,LI,TGVEG.mito
32082,3fbcd979-a862-45bd-9283-ca110a54b733,96252.0,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,0.00000,...,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,0.000000,LI,TGVEG.mito
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80414,Tg_mtDNA_R,NaN,NaN,NaN,NaN,0.000000,1.00000,0.941176,0.0030,1.00000,...,0.941176,0.0030,1.000000,0.695652,0.811765,0.0267,0.695652,1.000000,HI,TGA4.mito
80415,Tg_mtDNA_S,NaN,0.932292,0.936585,0.0251,0.932292,1.00000,0.975610,0.0010,1.00000,...,0.975610,0.0010,1.000000,0.947917,0.468293,0.0085,0.947917,1.000000,HI,TGA4.mito
80416,Tg_mtDNA_T,NaN,0.907895,0.706093,0.0297,0.907895,1.00000,0.985663,0.0008,1.00000,...,0.985663,0.0008,1.000000,0.790210,0.512545,0.0294,0.790210,1.000000,HI,TGA4.mito
80417,Tg_mtDNA_U,NaN,0.971698,0.703390,0.0127,0.971698,1.00000,0.994350,0.0006,1.00000,...,0.994350,0.0006,1.000000,0.585799,0.954802,0.0737,0.585799,1.000000,HI,TGA4.mito
